In [94]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('/Users/jvonderhoff/Desktop/FantasyFootball/2024/prod_bestball_2024.db')

query = "SELECT * FROM lineups"
df = pd.read_sql(query, conn)

In [95]:
unique_lineup_ids = df['lineup_ID'].nunique()
print("Number of unique lineup_ID values:", unique_lineup_ids)


Number of unique lineup_ID values: 19


In [96]:
unique_count_per_site = df.groupby('contest_site')['lineup_ID'].nunique()
print(unique_count_per_site)

contest_site
Draftkings    17
Underdog       2
Name: lineup_ID, dtype: int64


In [97]:
unique_count_per_site_name = df.groupby(['contest_site', 'contest_name'])['lineup_ID'].nunique()
print(unique_count_per_site_name)

contest_site  contest_name   
Draftkings    Best Ball Milly     2
              Play Action        15
Underdog      The Poodle          2
Name: lineup_ID, dtype: int64


# Money Spent
Underdog credit
1. 25 dollar deposit Bonus
2. 25 dollar Legup Credit

DraftKings Credit
1. 1 free entry into Milly

Money Spent
1. 99 dollars for a year of legup
2. 4.99 a month started beginning of May

In [98]:
total_spent_per_lineup = df.drop_duplicates('lineup_ID')['contest_entryFee'].sum()
print(total_spent_per_lineup)

99


In [99]:
total_spent_per_lineup_site = df.drop_duplicates('lineup_ID').groupby(['lineup_ID', 'contest_site'])['contest_entryFee'].sum().reset_index()
total_spent_per_site = total_spent_per_lineup_site.groupby('contest_site')['contest_entryFee'].sum()
print(total_spent_per_site)

contest_site
Draftkings    85
Underdog      14
Name: contest_entryFee, dtype: int64


In [100]:
total_spent_per_lineup = df.drop_duplicates('lineup_ID').groupby(['lineup_ID', 'contest_name'])['contest_entryFee'].sum().reset_index()
total_spent_per_contest = total_spent_per_lineup.groupby('contest_name')['contest_entryFee'].sum()
print(total_spent_per_contest)

contest_name
Best Ball Milly    40
Play Action        45
The Poodle         14
Name: contest_entryFee, dtype: int64


# Lineup Analysis

In [102]:
# Specify the desired player
desired_player = "Ja'Marr Chase"

# Filter lineups that include the desired player
filtered_lineups = df[df['player_name'] == desired_player]

# Print the lineup combinations for teams with the specific player
for lineup_id in filtered_lineups['lineup_ID'].unique():
    lineup = df[df['lineup_ID'] == lineup_id]['player_name'].tolist()
    print(f"Lineup {lineup_id}: {lineup}")

Lineup 7503db21-be71-4e4d-ba2b-e47bbcff570b: ["Ja'Marr Chase", 'Nico Collins', 'Cooper Kupp', 'Tee Higgins', 'Dalton Kincaid', 'Kenneth Walker III', 'George Kittle', 'Diontae Johnson', 'Tony Pollard', 'Devin Singletary', 'Matthew Stafford', 'Ezekiel Elliott', 'Jerome Ford', "Ja'Lynn Polk", 'Drake Maye', 'Marvin Mims Jr.', 'Bo Nix', 'Malik Washington', 'Dylan Laube', 'Jacoby Brissett']
Lineup d1729559-b42d-4533-85cc-742ba17b5d9c: ["Ja'Marr Chase", 'Chris Olave', 'Malik Nabers', 'Tee Higgins', 'Tank Dell', 'Rome Odunze', 'Kyler Murray', 'Jaylen Warren', 'Trey Benson', 'Brian Robinson Jr.', 'Chase Brown', 'Jerome Ford', 'Deshaun Watson', 'Michael Wilson', 'Xavier Legette', 'Bryce Young', 'Tyler Conklin', 'Noah Fant', 'Mike Gesicki', 'Dawson Knox']
Lineup a837181d-86db-4209-aaa1-c82156af5318: ["Ja'Marr Chase", 'Brandon Aiyuk', 'Deebo Samuel Sr.', 'Jalen Hurts', 'Trey McBride', 'Joe Burrow', 'Jordan Addison', 'Najee Harris', 'Javonte Williams', 'Courtland Sutton', 'Brian Robinson Jr.', 'Jer

# Statistics

In [77]:
def calculate_jaccard_similarity(lineup1, lineup2):
    set1 = set(lineup1)
    set2 = set(lineup2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    jaccard_similarity = intersection / union
    return jaccard_similarity

# Extract player sets for two lineups (e.g., lineup 1 and lineup 2)
lineup1 = df.loc[df['lineup_ID'] == '7503db21-be71-4e4d-ba2b-e47bbcff570b', 'player_name'].tolist()
lineup2 = df.loc[df['lineup_ID'] == '8b15b096-bb27-46d8-8723-3b2dd8702d6d', 'player_name'].tolist()

# Calculate the Jaccard similarity
similarity = calculate_jaccard_similarity(lineup1, lineup2)
print("Jaccard Similarity:", similarity)

Jaccard Similarity: 0.05263157894736842


In [78]:
def calculate_jaccard_similarity(lineup1, lineup2):
    set1 = set(lineup1)
    set2 = set(lineup2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    jaccard_similarity = intersection / union
    return jaccard_similarity

# Extract unique lineup IDs
lineup_ids = df['lineup_ID'].unique()

# Calculate Jaccard similarity for all lineup pairs
similarities = []
for i in range(len(lineup_ids)):
    for j in range(i+1, len(lineup_ids)):
        lineup1 = df.loc[df['lineup_ID'] == lineup_ids[i], 'player_name'].tolist()
        lineup2 = df.loc[df['lineup_ID'] == lineup_ids[j], 'player_name'].tolist()
        similarity = calculate_jaccard_similarity(lineup1, lineup2)
        similarities.append((lineup_ids[i], lineup_ids[j], similarity))

# Convert similarities to DataFrame for easier analysis
similarities_df = pd.DataFrame(similarities, columns=['lineup1', 'lineup2', 'similarity'])
print(similarities_df)

                                  lineup1  \
0    7503db21-be71-4e4d-ba2b-e47bbcff570b   
1    7503db21-be71-4e4d-ba2b-e47bbcff570b   
2    7503db21-be71-4e4d-ba2b-e47bbcff570b   
3    7503db21-be71-4e4d-ba2b-e47bbcff570b   
4    7503db21-be71-4e4d-ba2b-e47bbcff570b   
..                                    ...   
166  68d00616-d981-4cdd-adf3-46c0d198bd26   
167  68d00616-d981-4cdd-adf3-46c0d198bd26   
168  5800f56c-8272-4fb5-8697-8db05217b39a   
169  5800f56c-8272-4fb5-8697-8db05217b39a   
170  00c541c0-d0f3-41fb-8d5d-04d7618bcd23   

                                  lineup2  similarity  
0    8b15b096-bb27-46d8-8723-3b2dd8702d6d    0.052632  
1    c73fa1b1-97e1-429f-887f-8f6fa114817b    0.081081  
2    92d5fe6e-662b-4e09-ad1d-70c834cfb373    0.025641  
3    d1729559-b42d-4533-85cc-742ba17b5d9c    0.081081  
4    c64c8ad8-dbb4-4e9e-bf88-43ca89282be6    0.000000  
..                                    ...         ...  
166  00c541c0-d0f3-41fb-8d5d-04d7618bcd23    0.027027  
167  304ba0

In [79]:
# Extract unique lineup IDs
lineup_ids = df['lineup_ID'].unique()

# Calculate Jaccard similarity for all lineup pairs
similarities = []
for i in range(len(lineup_ids)):
    for j in range(i+1, len(lineup_ids)):
        lineup1 = df.loc[df['lineup_ID'] == lineup_ids[i], 'player_name'].tolist()
        lineup2 = df.loc[df['lineup_ID'] == lineup_ids[j], 'player_name'].tolist()
        similarity = calculate_jaccard_similarity(lineup1, lineup2)
        similarities.append((lineup_ids[i], lineup_ids[j], similarity))

# Convert similarities to DataFrame for easier analysis
similarities_df = pd.DataFrame(similarities, columns=['lineup1', 'lineup2', 'similarity'])

# Sort the DataFrame by similarity in descending order
similarities_df = similarities_df.sort_values(by='similarity', ascending=False)

# Print the top similar lineups and their shared players
num_top_lineups = 5  # Specify the number of top similar lineups you want to analyze
for index, row in similarities_df.head(num_top_lineups).iterrows():
    lineup1 = df.loc[df['lineup_ID'] == row['lineup1'], 'player_name'].tolist()
    lineup2 = df.loc[df['lineup_ID'] == row['lineup2'], 'player_name'].tolist()
    shared_players = list(set(lineup1) & set(lineup2))
    print(f"Lineup {row['lineup1']} and Lineup {row['lineup2']} have a Jaccard similarity of {row['similarity']}:")
    print(f"Shared Players: {shared_players}")
    print("-------")

Lineup a837181d-86db-4209-aaa1-c82156af5318 and Lineup 8b584c8c-d33f-4f06-914c-84a900f736b4 have a Jaccard similarity of 0.25:
Shared Players: ['Deebo Samuel Sr.', 'Brandon Aiyuk', 'Jermaine Burton', 'Jalen Hurts', 'Hunter Henry', 'Brian Robinson Jr.', 'Courtland Sutton', 'Joe Burrow']
-------
Lineup d1729559-b42d-4533-85cc-742ba17b5d9c and Lineup fad11e73-c167-4db0-ab18-ecf6a98d339b have a Jaccard similarity of 0.17647058823529413:
Shared Players: ['Trey Benson', 'Malik Nabers', 'Rome Odunze', 'Dawson Knox', 'Jaylen Warren', 'Noah Fant']
-------
Lineup 341ae308-9744-4346-be5b-71f8404ca642 and Lineup 68d00616-d981-4cdd-adf3-46c0d198bd26 have a Jaccard similarity of 0.17647058823529413:
Shared Players: ['Christian McCaffrey', 'Aaron Rodgers', 'Bryce Young', 'Deebo Samuel Sr.', 'Christian Kirk', 'Kalif Raymond']
-------
Lineup 7ecab08b-04e3-4faf-8621-388459ab077e and Lineup 5800f56c-8272-4fb5-8697-8db05217b39a have a Jaccard similarity of 0.17647058823529413:
Shared Players: ['Ezekiel El